In [ ]:
import os
import requests
import json 
from dotenv import load_dotenv
import time
load_dotenv()
import pandas as pd 
import csv
import tweepy
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import re
import numpy as np


In [ ]:
# Setup API Keys
twitter_api_key = os.getenv("TWITTER_API_KEY")
twitter_secret_key = os.getenv("TWITTER_SECRET_KEY")
twitter_access_token = os.getenv("TWITTER_ACCESS_TOKEN")
twitter_secret_token = os.getenv("TWITTER_SECRET_TOKEN")

# Authorize Account + create tweepy object
def oAuth():
    try:
        auth = tweepy.OAuthHandler(twitter_api_key,twitter_secret_key)
        auth.set_access_token(twitter_access_token,twitter_secret_token)
        return auth
    except Exception as e:
        return None

oauth= oAuth()

tweepy_api = tweepy.API(oauth)

# Create function to pull tweets 
screen_names = ['Enter_Twitter_Account_Here']

def get_new_tweets(names):
    print("Retrieving tweets")
    corpus = []                                                                                        
    for name in names:
        tweets = tweepy_api.user_timeline(screen_name = name, include_rts=False, count=5, tweet_mode="extended", exclude_replies = True)          
        time.sleep(4)
        corpus.extend(tweets)                                                                          
    data = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at] for tweet in corpus]
    tweets = pd.DataFrame(data, columns=['tweet_id', 'screen_name', 'text', 'timestamp'])                

    return tweets

user_tweets = get_new_tweets(screen_names)

# Gather replies to the chosen tweet:
replies=[]
for tweet in tweepy.Cursor(tweepy_api.search_tweets,q='to:'+name, result_type = 'recent', tweet_mode = 'extended').items(100):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

# Create a Dataframe:
text = []

for i in range(len(replies)):
    text.append(replies[i].full_text)

df = pd.DataFrame(
    {'text': text,})

# Clean the tweets:
def clean_tweets(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www.\S+", "", text)
    
    return text

df['clean_text'] = df['text'].apply(clean_tweets)
df = df.drop('text',axis=1)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f" 
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

df['clean_text'] = df['clean_text'].apply(remove_emojis)

# Write to excel file:
with pd.ExcelWriter('YOUR_EXCEL_DOC_LOCATION',engine= 'openpyxl', mode= 'a', if_sheet_exists='new') as writer:
    df.to_excel(writer)